In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# This model works optimally around 100-200 epochs before it diverges 

In [2]:
class GAN:
    def __init__(self, latent_dim, data_dim):
        # Initialize GAN parameters
        self.latent_dim = latent_dim  # Dimension of the noise vector
        self.data_dim = data_dim  # Dimension of the data

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # Define the input for the generator
        z = Input(shape=(self.latent_dim,))
        generated_data = self.generator(z)

        # For the combined model, we will only train the generator
        self.discriminator.trainable = False

        # Define the validity of the generated data through the discriminator
        validity = self.discriminator(generated_data)

        # Define and compile the combined model (generator stacked with discriminator)
        self.combined = Model(z, validity)
        self.combined.compile(
            loss='binary_crossentropy',
            optimizer=Adam(learning_rate=0.0001, beta_1=0.5)
        )

        # Reset the discriminator's trainable attribute to True
        self.discriminator.trainable = True

    def build_generator(self):
        # Build the generator model
        model = Sequential()
        model.add(Input(shape=(self.latent_dim,)))
        model.add(Dense(256))
        model.add(LeakyReLU(negative_slope=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(negative_slope=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(negative_slope=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.data_dim, activation='tanh'))
        return model

    def build_discriminator(self):
        # Build the discriminator model
        model = Sequential()
        model.add(Input(shape=(self.data_dim,)))
        model.add(Dense(512))
        model.add(LeakyReLU(negative_slope=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(negative_slope=0.2))
        model.add(Dense(1, activation='sigmoid'))
        return model

    def train(self, data, epochs, batch_size):
        # Rescale the data to the range [-1, 1]
        data = data * 2 - 1
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            idx = np.random.randint(0, data.shape[0], batch_size)
            real_data = data[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            generated_data = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(real_data, valid)
            d_loss_fake = self.discriminator.train_on_batch(generated_data, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)

            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")


In [3]:
# Load CSV files and merge them
ads_data_path = r'C:\Users\lucas\Downloads\GES Hackathon\decrypted_file\train\train_data_ads.csv'  # Replace with actual file path
feeds_data_path = r'C:\Users\lucas\Downloads\GES Hackathon\decrypted_file\train\train_data_feeds.csv'  # Replace with actual file path

ads_data = pd.read_csv(ads_data_path)
feeds_data = pd.read_csv(feeds_data_path)

In [4]:
# Fractions of data used as loading both of them in full raises a memory allocation error due to large size
sample_ads_data = ads_data.sample(frac=.01, random_state=42)
sample_feeds_data = feeds_data.sample(frac=.01, random_state=42)

In [5]:
click = pd.merge(sample_ads_data, sample_feeds_data, left_on='user_id', right_on='u_userId')

In [6]:
click.head()

,log_id,label_x,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label_y,cillabel,pro
0,389752,0,192266,2,2,46,354,2,11,8,...,19,320,2,214,2,0,202206080822,-1,-1,0
1,389752,0,192266,2,2,46,354,2,11,8,...,19,320,9,214,6,0,202206032022,-1,-1,0
2,237151,0,231175,6,4,33,319,3,27,2,...,19,1185,16,1830,1,0,202206051726,-1,-1,0
3,963804,0,203833,8,2,30,113,5,31,3,...,19,565,10,1396,57,0,202206031220,-1,-1,0
4,376817,0,115573,2,2,27,162,5,30,3,...,19,327,7,2073,2,0,202206070922,1,-1,100


In [7]:
size = 5  # Every list seperated by ^ is at max 5 long 

def normalize_list_length(lst, max_length):
    if len(lst) > max_length:  # If the list is longer than the fixed length
        return lst[:max_length]  # Truncate the list
    else:  # If the list is shorter than the fixed length
        return lst + ['0'] * (max_length - len(lst))

def split_list_column(df, column_name, max_length):
    # Ensure the column contains lists before processing
    if df[column_name].apply(lambda x: isinstance(x, list)).all():
        # Create a DataFrame from the lists with appropriate column names
        split_columns = pd.DataFrame(df[column_name].tolist(),
                                     columns=[f'{column_name}_{i+1}' for i in range(max_length)])
        # Drop the original column and join the new columns
        df = df.drop(columns=[column_name]).join(split_columns)
    return df


for col in click.columns: # This creates necessary columns 
    if click[col].dtype == object:  
        max_length = click[col].apply(lambda x: x.count('^') + 1 if isinstance(x, str) else 1).max() 
        click[col] = click[col].apply(lambda x: x.split('^') if isinstance(x, str) else x)
        click[col] = click[col].apply(lambda x: normalize_list_length(x, max_length) if isinstance(x, list) else x)
        click = split_list_column(click, col, max_length)

click = click.drop(columns=['i_entities', 'i_docId_1', 'i_s_sourceId_1']) # Drop strings for now
click.head()


,log_id,label_x,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,u_newsCatInterestsST_y_1,u_newsCatInterestsST_y_2,u_newsCatInterestsST_y_3,u_newsCatInterestsST_y_4,u_newsCatInterestsST_y_5,u_click_ca2_news_1,u_click_ca2_news_2,u_click_ca2_news_3,u_click_ca2_news_4,u_click_ca2_news_5
0,389752,0,192266,2,2,46,354,2,11,8,...,85,65,152,116,177,85,152,65,116,25
1,389752,0,192266,2,2,46,354,2,11,8,...,173,50,123,0,0,173,50,123,80,114
2,237151,0,231175,6,4,33,319,3,27,2,...,112,109,44,153,157,112,17,109,78,153
3,963804,0,203833,8,2,30,113,5,31,3,...,168,167,65,0,0,168,167,219,65,109
4,376817,0,115573,2,2,27,162,5,30,3,...,199,25,171,65,104,65,104,131,199,66


In [8]:
for column in click.columns: # Convert to integer
    if click[column].dtype == object:
        click[column] = click[column].astype(int)

In [9]:
scaler = MinMaxScaler(feature_range=(-1, 1))  # Use MinMaxScaler for normalization

# Normalize the data
click_array = scaler.fit_transform(click.values)

# Define GAN parameters
latent_dim = 100
data_dim = click_array.shape[1]

# Instantiate and train the GAN
gan = GAN(latent_dim, data_dim)
gan.train(click_array, epochs=200, batch_size=128)  # Choose the amount of runs 

# Generate synthetic data
num_samples = 10000  # Adjust the number of samples as needed
noise = np.random.normal(0, 1, (num_samples, latent_dim))
synthetic_data = gan.generator.predict(noise)

# Inverse transform the synthetic data
synthetic_data = scaler.inverse_transform(synthetic_data)

# Handle NaN and infinite values before rounding
synthetic_data = np.nan_to_num(synthetic_data, nan=0.0, posinf=0.0, neginf=0.0)

# Clip the data to a reasonable range, e.g., [0, max_value]
max_value = click.max().max()  # Get max of numeric columns
synthetic_data = np.clip(synthetic_data, 0, max_value)

# Round the data and convert to integers
synthetic_data = np.round(synthetic_data).astype(int)

# Convert synthetic data back to a DataFrame
synthetic_click = pd.DataFrame(synthetic_data, columns=click.columns)

# Debug: Check the shape and head of the generated data
print(f"Shape of generated data: {synthetic_click.shape}")
synthetic_click.head()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,137 (106.00 KB)

 Trainable params: 27,137 (106.00 KB)

 Non-trainable params: 0 (0.00 B)

...

 Trainable params: 53,096 (207.41 KB)

 Non-trainable params: 512 (2.00 KB)

Discriminator trainable weights: [<KerasVariable shape=(104, 256), dtype=float32, path=sequential/dense/kernel>, <KerasVariable shape=(256,), dtype=float32, path=sequential/dense/bias>, <KerasVariable shape=(256, 1), dtype=float32, path=sequential/dense_1/kernel>, <KerasVariable shape=(1,), dtype=float32, path=sequential/dense_1/bias>]
Generator trainable weights: [<KerasVariable shape=(100, 256), dtype=float32, path=sequential_1/dense_2/kernel>, <KerasVariable shape=(256,), dtype=float32, path=sequential_1/dense_2/bias>, <KerasVariable shape=(256,), dtype=float32, path=sequential_1/batch_normalization/gamma>, <KerasVariable shape=(256,), dtype=float32, path=sequential_1/batch_normalization/beta>, <KerasVariable shape=(256, 104), dtype=float32, path=sequential_1/dense_3/kernel>, <KerasVariable shape=(104,), dtype=float32, path=sequential_1/dense_3/bias>]
Combined model trainable weights: [<KerasVariable shape=(100, 256), dtype=float32, path=sequential_1/dense_2/kernel>, <KerasVariable 

C:\Users\lucas\AppData\Local\Temp\ipykernel_12200\1585377447.py:30: RuntimeWarning: invalid value encountered in cast
  synthetic_data = np.round(synthetic_data).astype(int)


,log_id,label_x,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,u_newsCatInterestsST_y_1,u_newsCatInterestsST_y_2,u_newsCatInterestsST_y_3,u_newsCatInterestsST_y_4,u_newsCatInterestsST_y_5,u_click_ca2_news_1,u_click_ca2_news_2,u_click_ca2_news_3,u_click_ca2_news_4,u_click_ca2_news_5
0,796099,1,273183,7,3,32,380,4,26,7,...,150,208,218,134,144,214,142,208,118,200
1,1066403,1,250033,6,4,31,409,4,35,7,...,188,158,164,193,138,147,218,145,126,143
2,660569,1,266881,8,3,43,422,5,31,7,...,189,122,116,178,190,165,201,133,149,141
3,673197,1,259784,7,3,44,301,4,26,7,...,159,134,118,193,142,171,123,172,163,153
4,760501,1,251063,7,4,37,389,4,28,5,...,162,193,117,187,188,175,208,149,215,153


In [10]:
click.describe()

,log_id,label_x,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,u_newsCatInterestsST_y_1,u_newsCatInterestsST_y_2,u_newsCatInterestsST_y_3,u_newsCatInterestsST_y_4,u_newsCatInterestsST_y_5,u_click_ca2_news_1,u_click_ca2_news_2,u_click_ca2_news_3,u_click_ca2_news_4,u_click_ca2_news_5
count,7.310400e+04,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,...,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000,73104.000000
mean,5.456953e+05,0.008632,193422.610185,5.472710,2.375602,26.585248,268.858352,3.404643,24.455666,4.586042,...,122.068765,110.355753,105.502339,103.364385,102.512735,114.086001,111.556358,109.599653,108.906298,107.884822
std,3.181061e+05,0.092505,54052.268470,2.173391,0.735787,9.272219,98.773152,1.284477,7.760142,1.942284,...,65.173354,63.849116,62.077208,62.371114,61.594837,65.250141,64.075987,63.348185,64.481685,64.559538
min,4.000000e+00,0.000000,100022.000000,2.000000,2.000000,11.000000,101.000000,2.000000,11.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.688722e+05,0.000000,145387.000000,3.000000,2.000000,20.000000,175.000000,2.000000,16.000000,3.000000,...,78.000000,65.000000,62.000000,57.000000,57.000000,65.000000,63.000000,62.000000,57.000000,56.000000
50%,5.425155e+05,0.000000,192789.000000,6.000000,2.000000,26.000000,297.000000,3.000000,27.000000,5.000000,...,112.000000,98.000000,98.000000,98.000000,98.000000,108.000000,104.000000,100.000000,104.000000,104.000000
75%,8.176330e+05,0.000000,240004.000000,7.000000,2.000000,33.000000,342.000000,5.000000,31.000000,6.000000,...,171.000000,168.000000,157.000000,155.000000,152.000000,171.000000,168.000000,168.000000,168.000000,168.000000
max,1.176443e+06,1.000000,287127.000000,9.000000,4.000000,46.000000,441.000000,5.000000,37.000000,8.000000,...,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000,220.000000


In [11]:
synthetic_click.describe()

,log_id,label_x,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,u_newsCatInterestsST_y_1,u_newsCatInterestsST_y_2,u_newsCatInterestsST_y_3,u_newsCatInterestsST_y_4,u_newsCatInterestsST_y_5,u_click_ca2_news_1,u_click_ca2_news_2,u_click_ca2_news_3,u_click_ca2_news_4,u_click_ca2_news_5
count,1.000000e+04,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000
mean,8.715241e+05,1.0,239184.753700,7.256700,3.453000,37.269300,352.789500,4.293300,30.23790,6.477100,...,163.521700,166.880700,160.502100,161.753600,162.527900,163.484700,163.110600,168.338500,163.5000,163.659800
std,1.592501e+05,0.0,23386.801478,0.977395,0.497811,4.878766,44.742566,0.455298,3.42457,0.847672,...,27.988567,27.560857,28.640772,28.211237,26.170524,31.603434,31.849842,25.931555,29.1469,25.964104
min,5.892950e+05,1.0,194224.000000,6.000000,3.000000,29.000000,272.000000,4.000000,24.00000,5.000000,...,111.000000,111.000000,110.000000,110.000000,111.000000,110.000000,110.000000,111.000000,110.0000,112.000000
25%,7.350525e+05,1.0,219697.750000,6.000000,3.000000,33.000000,314.000000,4.000000,27.00000,6.000000,...,140.000000,143.000000,136.000000,138.000000,141.000000,135.000000,134.000000,148.000000,138.0000,142.000000
50%,8.665285e+05,1.0,238713.000000,7.000000,3.000000,37.000000,351.000000,4.000000,30.00000,6.000000,...,163.000000,168.000000,159.000000,161.000000,161.000000,163.000000,162.000000,169.000000,163.0000,163.000000
75%,1.006610e+06,1.0,258434.500000,8.000000,4.000000,41.000000,391.000000,5.000000,33.00000,7.000000,...,187.000000,190.000000,184.000000,185.000000,184.000000,191.250000,191.000000,190.000000,188.0000,185.000000
max,1.176057e+06,1.0,286644.000000,9.000000,4.000000,46.000000,440.000000,5.000000,37.00000,8.000000,...,219.000000,220.000000,219.000000,219.000000,219.000000,220.000000,220.000000,219.000000,220.0000,218.000000


In [12]:
click.to_csv(r'C:\Users\lucas\Downloads\GES Hackathon\decrypted_file\train\real_click.csv', index=False)
synthetic_click.to_csv(r'C:\Users\lucas\Downloads\GES Hackathon\decrypted_file\train\synthetic_click.csv', index=False)